In [34]:
import sys
import sdic
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from keras.layers import Dense, Dropout, Flatten, GaussianNoise, ZeroPadding2D, Input, concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential, Model, load_model
import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from pickle import load
from keras.preprocessing.image import ImageDataGenerator

In [15]:
def myGlobal0_to_1_Normalization(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [16]:
X = pd.read_csv("/workspace/DeepImageAI/expression_binary_autosome.csv")
Y = pd.read_csv("/workspace/DeepImageAI/input_meta_filtered_binray.csv")
Y = np.asarray(Y["Activity"])
X = (X.iloc[:, 1:].values).T
assert(X.shape[0] == Y.shape[0]) # Make sure we have the same number of samples

In [17]:
print(X.shape) # 550 samples and 33153 genes
print(Y.shape)

(550, 33153)
(550,)


In [18]:
P = 183
P * P
img_size = P
img_rows = P
img_cols = P

In [19]:
X1 = myGlobal0_to_1_Normalization(X) # MODE_ASIS
X1 = np.resize(X1, (550, P*P))
X1 = X1.reshape((550, P, P))
print(X1.shape) # Just like the code!
print(np.min(X1))
print(np.max(X1))

(550, 183, 183)
0.0
1.0


In [20]:
x_train = X
x_train = np.resize(x_train, (550, P*P))
x_train = x_train.reshape((550, P, P))

x_train_new_rand = np.zeros((x_train.shape[0], img_size, img_size))
#x_test_new_rand=np.zeros((x_test.shape[0],img_size,img_size))
ind=list(range(0,img_size*img_size))
np.random.seed(1234)
np.random.shuffle(ind)
cx=cy=0
dir=1
for i in range(0,img_size*img_size):
    dy=ind[i]//img_size
    dx=ind[i]%img_size
    x_train_new_rand[:,cy,cx]=x_train[:,dy,dx] 
#    x_test_new_rand[:,cy,cx]=x_test[:,dy,dx] 
    if dir==1: cx+=1
    else: cx-=1
    if cx==img_size:
        cy+=1
        cx-=1
        dir*=-1
    elif cx==-1:
        cy+=1
        cx+=1
        dir*=-1
    x_train_new=x_train_new_rand
    #x_test_new=x_test_new_rand
    
print(x_train.shape)
print(x_train_new.shape)
print(np.array_equal(x_train, x_train))
print(np.array_equal(x_train, x_train_new))
X2 = x_train_new # MODE_RAND

(550, 183, 183)
(550, 183, 183)
True
False


In [21]:
print(X.shape)
print(Y.shape)
X = myGlobal0_to_1_Normalization(X)

x_train = X
x_train = np.resize(x_train, (550, P*P))
x_train = x_train.reshape((550, P, P))

vic = sdic.sdic(sdic.SDIC_TYPE_SDIC)
vic.fit(x_train)
x_train_new = vic.transform(x_train)
X3 = x_train_new

(550, 33153)
(550,)


In [22]:
#
# Let's just train on the SDIC method for now!
#

1.0


In [24]:
x_train = X3
print(np.min(x_train))
print(np.max(x_train))
#x_train = myGlobal0_to_1_Normalization(x_train)
#print(np.min(x_train))
#print(np.max(x_train))

0.0
1.0


In [25]:
print(P)
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], 184, 184, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (184, 184, x_train.shape[3])
print(input_shape) # One channel!

183
(550, 184, 184)
(184, 184, 1)


In [26]:
y_train = Y
print(x_train.shape)
print(y_train.shape)

(550, 184, 184, 1)
(550,)


In [43]:
import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
#input_shape=np.shape(input_data_transformed)[1:4]

def createModel():
    input_img = Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(input_img)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x) 
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(x)    
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)    
    x = tf.keras.layers.Dropout(0.5)(x)    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)    
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='visualized_layer')(x)

    def auc(y_true, y_pred):
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)
    
    model = Model(inputs=input_img, outputs=output)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_crossentropy'])    
    return model

def newModel():
    model = Sequential()
    ks=(3,3)
    num_classes = 2
    model.add(Conv2D(32, kernel_size=(ks), activation='relu',input_shape=input_shape))
    model.add(Conv2D(64, ks, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) # Binary in NYG
    model.compile(loss=keras.losses.categorical_crossentropy if num_classes>2 else keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [51]:
model = createModel()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 184, 184, 1)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 182, 182, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 91, 91, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 91, 91, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 89, 89, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 44, 44, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 44, 44, 64)       

In [70]:
datagen = ImageDataGenerator()
X = x_train
Y = y_train
del x_train
del y_train

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.25, shuffle=True)

In [83]:
train_iterator = datagen.flow(X_train, Y_train, batch_size=32, seed=17, shuffle=False)
test_iterator  = datagen.flow(X_test,  Y_test,  batch_size=32, seed=17, shuffle=False)

In [84]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_train),
                                                 Y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
print(Y_train.shape)

{0: 4.5777777777777775, 1: 0.5613079019073569}
(412,)


In [85]:
checkpoint_path_incep = "Binary_all_vs_control/cp_incep.ckpt"
checkpoint_dir_incep = os.path.dirname(checkpoint_path_incep)

from tensorflow.python.keras.callbacks import ModelCheckpoint
callbacks_list = [   
    ModelCheckpoint(filepath=checkpoint_path_incep, monitor='val_loss', save_best_only=True, save_weights_only=True,verbose=1, mode='min')
]

In [86]:
print(X_train.shape)
print(X_test.shape)

(412, 184, 184, 1)
(138, 184, 184, 1)


In [87]:
with tf.device('/gpu:0'):
    history = model.fit_generator(train_iterator,                           
                                  epochs=50,
                                  validation_data=test_iterator,
                                  #class_weight=d_class_weights,
                                  callbacks=callbacks_list)

Epoch 1/50
13/13 [==============================] - ETA: 0s - loss: 8.1939e-05 - binary_crossentropy: 8.1939e-05
Epoch 00001: val_loss improved from inf to 0.00006, saving model to Binary_all_vs_control/cp_incep.ckpt
13/13 [==============================] - 7s 509ms/step - loss: 8.1939e-05 - binary_crossentropy: 8.1939e-05 - val_loss: 5.7782e-05 - val_binary_crossentropy: 5.7782e-05
Epoch 2/50
13/13 [==============================] - ETA: 0s - loss: 5.3286e-05 - binary_crossentropy: 5.3286e-05
Epoch 00002: val_loss improved from 0.00006 to 0.00005, saving model to Binary_all_vs_control/cp_incep.ckpt
13/13 [==============================] - 7s 506ms/step - loss: 5.3286e-05 - binary_crossentropy: 5.3286e-05 - val_loss: 4.9562e-05 - val_binary_crossentropy: 4.9562e-05
Epoch 3/50
13/13 [==============================] - ETA: 0s - loss: 4.9823e-05 - binary_crossentropy: 4.9823e-05
Epoch 00003: val_loss improved from 0.00005 to 0.00004, saving model to Binary_all_vs_control/cp_incep.ckpt
13/

Epoch 22/50
13/13 [==============================] - ETA: 0s - loss: 2.0580e-05 - binary_crossentropy: 2.0580e-05
Epoch 00022: val_loss improved from 0.00001 to 0.00001, saving model to Binary_all_vs_control/cp_incep.ckpt
13/13 [==============================] - 7s 508ms/step - loss: 2.0580e-05 - binary_crossentropy: 2.0580e-05 - val_loss: 8.2178e-06 - val_binary_crossentropy: 8.2178e-06
Epoch 23/50
13/13 [==============================] - ETA: 0s - loss: 0.0111 - binary_crossentropy: 0.0111
Epoch 00023: val_loss did not improve from 0.00001
13/13 [==============================] - 5s 349ms/step - loss: 0.0111 - binary_crossentropy: 0.0111 - val_loss: 5.6719e-04 - val_binary_crossentropy: 5.6719e-04
Epoch 24/50
13/13 [==============================] - ETA: 0s - loss: 9.8215e-04 - binary_crossentropy: 9.8215e-04
Epoch 00024: val_loss did not improve from 0.00001
13/13 [==============================] - 5s 350ms/step - loss: 9.8215e-04 - binary_crossentropy: 9.8215e-04 - val_loss: 8.1053

Epoch 47/50
13/13 [==============================] - ETA: 0s - loss: 1.3345e-05 - binary_crossentropy: 1.3345e-05
Epoch 00047: val_loss did not improve from 0.00001
13/13 [==============================] - 5s 351ms/step - loss: 1.3345e-05 - binary_crossentropy: 1.3345e-05 - val_loss: 1.6828e-05 - val_binary_crossentropy: 1.6828e-05
Epoch 48/50
13/13 [==============================] - ETA: 0s - loss: 8.3484e-06 - binary_crossentropy: 8.3484e-06
Epoch 00048: val_loss did not improve from 0.00001
13/13 [==============================] - 5s 354ms/step - loss: 8.3484e-06 - binary_crossentropy: 8.3484e-06 - val_loss: 1.5272e-05 - val_binary_crossentropy: 1.5272e-05
Epoch 49/50
13/13 [==============================] - ETA: 0s - loss: 9.6057e-06 - binary_crossentropy: 9.6057e-06
Epoch 00049: val_loss did not improve from 0.00001
13/13 [==============================] - 5s 354ms/step - loss: 9.6057e-06 - binary_crossentropy: 9.6057e-06 - val_loss: 1.4645e-05 - val_binary_crossentropy: 1.4645e-0

In [88]:
print("Completed!")

Completed!


In [89]:
model_incep_loaded = createModel()

In [90]:
model_incep_loaded.load_weights(checkpoint_path_incep)

In [91]:
pred_train_inception_load = model_incep_loaded.predict_generator(train_iterator, steps=len(train_iterator), verbose=0)

In [93]:
auc_train_external_stack_pos = roc_auc_score(Y_train, pred_train_inception_load)
print("auc_train_external_stack_pos: " + str(auc_train_external_stack_pos))
pred_train_external_stack_pos = np.where(pred_train_inception_load > 0.5, 1, 0)
tn, fp, fn, tp = confusion_matrix(Y_train, pred_train_external_stack_pos).ravel()
specificity_train_external_stack_pos = tn / (tn + fp)
sensitivity_train_external_stack_pos = tp / (tp + fn)
NPV_train_external_stack_pos = tn / (tn + fn)
PPV_train_external_stack_pos = tp / (tp + fp)
Accuracy_train_external_stack_pos = balanced_accuracy_score(Y_train, pred_train_external_stack_pos)
MCC_train_external_stack_pos = matthews_corrcoef(Y_train, pred_train_external_stack_pos)
print("specificity_train_external_stack_pos: " + str(specificity_train_external_stack_pos))
print("sensitivity_train_external_stack_pos: " + str(sensitivity_train_external_stack_pos))
print("NPV_train_external_stack_pos: " + str(NPV_train_external_stack_pos))
print("PPV_train_external_stack_pos: " + str(PPV_train_external_stack_pos))
print("Accuracy_train_external_stack_pos: " + str(Accuracy_train_external_stack_pos))
print("MCC_train_external_stack_pos: " + str(MCC_train_external_stack_pos))

from sklearn.metrics import f1_score
print("F1 SCORE train= "+str(f1_score(Y_train, pred_train_external_stack_pos, average='macro')))
print(classification_report(Y_train, pred_train_external_stack_pos))

auc_train_external_stack_pos: 1.0
specificity_train_external_stack_pos: 1.0
sensitivity_train_external_stack_pos: 1.0
NPV_train_external_stack_pos: 1.0
PPV_train_external_stack_pos: 1.0
Accuracy_train_external_stack_pos: 1.0
MCC_train_external_stack_pos: 1.0
F1 SCORE train= 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       1.00      1.00      1.00       367

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412



In [95]:
pred_test_inception_load = model_incep_loaded.predict_generator(test_iterator, steps=len(test_iterator), verbose=0)
auc_test_external_stack_pos = roc_auc_score(Y_test, pred_test_inception_load)
print("auc_test_external_stack_pos: " + str(auc_test_external_stack_pos))
pred_test_external_stack_pos = np.where(pred_test_inception_load > 0.5, 1, 0)
tn, fp, fn, tp = confusion_matrix(Y_test, pred_test_external_stack_pos).ravel()
specificity_test_external_stack_pos = tn / (tn + fp)
sensitivity_test_external_stack_pos = tp / (tp + fn)
NPV_test_external_stack_pos = tn / (tn + fn)
PPV_test_external_stack_pos = tp / (tp + fp)
Accuracy_test_external_stack_pos = balanced_accuracy_score(Y_test, pred_test_external_stack_pos)
MCC_test_external_stack_pos = matthews_corrcoef(Y_test, pred_test_external_stack_pos)
print("specificity_test_external_stack_pos: " + str(specificity_test_external_stack_pos))
print("sensitivity_test_external_stack_pos: " + str(sensitivity_test_external_stack_pos))
print("NPV_test_external_stack_pos: " + str(NPV_test_external_stack_pos))
print("PPV_test_external_stack_pos: " + str(PPV_test_external_stack_pos))
print("Accuracy_test_external_stack_pos: " + str(Accuracy_test_external_stack_pos))
print("MCC_test_external_stack_pos: " + str(MCC_test_external_stack_pos))

from sklearn.metrics import f1_score
print("F1 SCORE test = "+str(f1_score(Y_test, pred_test_external_stack_pos, average='macro')))
print(classification_report(Y_test, pred_test_external_stack_pos))

auc_test_external_stack_pos: 1.0
specificity_test_external_stack_pos: 1.0
sensitivity_test_external_stack_pos: 1.0
NPV_test_external_stack_pos: 1.0
PPV_test_external_stack_pos: 1.0
Accuracy_test_external_stack_pos: 1.0
MCC_test_external_stack_pos: 1.0
F1 SCORE test = 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00       123

    accuracy                           1.00       138
   macro avg       1.00      1.00      1.00       138
weighted avg       1.00      1.00      1.00       138

